# 모델링을 위한 질문 생성 코드

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import json
import random
import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. 모델 설정 (가성비 좋은 gpt-5-nano 활용)
llm = ChatOpenAI(model="gpt-5-nano", temperature=0.7)

def generate_triplet_dataset(input_file, output_file, num_samples=10):
    # 데이터 로드
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            all_benefits = json.load(f)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {input_file}")
        return

    # 프롬프트 설정: 페르소나가 담긴 자연스러운 구어체 질문 유도
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 금융 도메인 특화 데이터 엔지니어입니다."),
        ("user", """아래 [혜택 정보]를 보고, 이 혜택을 필요로 하는 사용자가 던질 법한 아주 자연스러운 질문 1개를 생성하세요.

[혜택 정보]: {benefit_content}

조건:
1. 질문(anchor)은 "~살이고 ~하는 상황인데"와 같이 구체적인 상황과 페르소나가 느껴져야 합니다.
2. 질문은 카드 이름이나 상품명을 직접 언급하지 말고, '혜택의 필요성'에 집중하세요.
3. 결과는 반드시 아래 JSON 형식으로만 출력하세요.
{{
    "anchor": "생성된 구어체 질문",
    "positive": "제공된 혜택 정보 문구 그대로 유지"
}}
""")
    ])

    dataset = []
    
    # 학습 데이터 생성을 위한 샘플링 (테스트를 위해 num_samples만큼 진행)
    selected_benefits = all_benefits[:num_samples]
    print(f"🚀 총 {len(selected_benefits)}건의 혜택에 대해 질문 생성 및 데이터셋 구축 시작...")

    for i, benefit in enumerate(selected_benefits):
        content = benefit.get('content')
        metadata = benefit.get('metadata')
        
        # 1. LLM을 통한 Anchor(질문) 생성
        chain = prompt | llm
        try:
            response = chain.invoke({"benefit_content": content})
            ai_data = json.loads(response.content.strip().replace('```json', '').replace('```', ''))
            
            # 2. Negative 데이터 구성 (Hard Negative 전략)
            # 동일한 카드의 다른 카테고리 혜택을 우선적으로 오답으로 선택 (모델이 헷갈리게 만들기 위함)
            same_card_others = [
                b['content'] for b in all_benefits 
                if b['metadata']['card_id'] == metadata['card_id'] and b['metadata']['category'] != metadata['category']
            ]
            
            if same_card_others:
                negative_text = random.choice(same_card_others)
            else:
                # 다른 카드의 혜택을 무작위로 선택
                other_random_benefit = random.choice([b for b in all_benefits if b['metadata']['card_id'] != metadata['card_id']])
                negative_text = other_random_benefit['content']

            # 3. 최종 Triplet 구조 생성
            result_item = {
                "anchor": ai_data['anchor'],
                "positive": content, # 원래의 혜택 내용
                "negative": negative_text,
                "metadata": {
                    "category": metadata['category'],
                    "card_id": metadata['card_id'],
                    "card_name": metadata['card_name']
                }
            }
            
            dataset.append(result_item)
            print(f"✅ [{i+1}/{num_samples}] '{metadata['card_name']}'의 '{metadata['category']}' 데이터 생성 완료")
            
            # API 제한 방지
            time.sleep(0.5)

        except Exception as e:
            print(f"❌ 에러 발생 (Index {i}): {e}")

    # 결과 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)
    
    print(f"\n✨ 데이터셋 구축 완료! 파일명: {output_file}")

if __name__ == "__main__":
    # 신용카드 데이터를 먼저 시도해보세요.
    generate_triplet_dataset('all_credit_benefits_rag.json', 'sbert_train_dataset.json', num_samples=10)

In [ ]:
import asyncio
import json
import random
import nest_asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. 주피터 노트북 이벤트 루프 충돌 방지
nest_asyncio.apply()

# 2. 비동기 모델 및 프롬프트 설정
llm = ChatOpenAI(model="gpt-5-nano", temperature=0.7)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 카드 추천 챗봇의 학습 데이터를 생성하는 전문가입니다."),
    ("user", """아래 [혜택 정보]를 바탕으로 사용자가 물어볼 법한 자연스러운 질문 1개를 생성하세요.
    [혜택 정보]: {benefit_content}
    
    조건:
    1. 질문(anchor)은 "~살이고 ~한 상황인데"와 같이 구체적인 페르소나가 느껴져야 합니다.
    2. 결과는 반드시 아래 JSON 형식으로만 출력하세요.
    {{
        "anchor": "생성된 질문",
        "positive": "제공된 혜택 정보 문구 그대로 유지"
    }}
    """)
])

async def process_batch(batch_chunks, all_data):
    chain = prompt | llm
    tasks = [chain.ainvoke({"benefit_content": b['content']}) for b in batch_chunks]
    
    # return_exceptions=True로 설정하여 일부 요청 실패 시에도 전체가 멈추지 않게 함
    responses = await asyncio.gather(*tasks, return_exceptions=True)
    
    batch_results = []
    for idx, res in enumerate(responses):
        if isinstance(res, Exception):
            # Rate Limit(429) 에러 발생 시 로그 출력
            print(f"⚠️ 요청 실패 (ID {batch_chunks[idx]['metadata']['card_id']}): {res}")
            continue
            
        try:
            raw_content = res.content.strip().replace('```json', '').replace('```', '')
            ai_data = json.loads(raw_content)
            meta = batch_chunks[idx]['metadata']
            
            # Hard Negative 선택
            same_card_others = [
                item['content'] for item in all_data 
                if item['metadata']['card_id'] == meta['card_id'] and item['metadata']['category'] != meta['category']
            ]
            negative_text = random.choice(same_card_others) if same_card_others else "관련 없는 혜택 정보입니다."

            batch_results.append({
                "anchor": ai_data['anchor'],
                "positive": ai_data['positive'],
                "negative": negative_text,
                "metadata": {
                    "category": meta['category'],
                    "card_id": meta['card_id']
                }
            })
        except Exception as e:
            print(f"❌ 데이터 파싱 에러: {e}")
            
    return batch_results

async def main():
    input_file = 'all_credit_benefits_rag.json'
    output_file = 'final_sbert_dataset.json'
    
    with open(input_file, 'r', encoding='utf-8') as f:
        all_benefits = json.load(f)

    # 🚀 병렬 처리 속도 조절 (Rate Limit에 걸리면 10~15로 줄이세요)
    batch_size = 20 
    final_dataset = []
    
    print(f"🚀 총 {len(all_benefits)}건 처리 시작... (병렬 크기: {batch_size})")

    for i in range(0, len(all_benefits), batch_size):
        batch_chunks = all_benefits[i:i+batch_size]
        results = await process_batch(batch_chunks, all_benefits)
        final_dataset.extend(results)
        
        # 200개마다 진행 상황 저장 (주기 단축)
        if (i + batch_size) % 200 == 0:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(final_dataset, f, ensure_ascii=False, indent=2)
            print(f"💾 {i + batch_size}/{len(all_benefits)} 진행 중... (파일 저장 완료)")
        
        # API 서버 휴식 시간 (필수)
        await asyncio.sleep(0.5)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, ensure_ascii=False, indent=2)
    print(f"✨ 모든 작업 완료! 총 {len(final_dataset)}개 생성됨.")

# --- 주피터 실행부 ---
if __name__ == "__main__":
    # asyncio.run(main()) 대신 await를 직접 사용하거나 nest_asyncio와 함께 호출
    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())
    except Exception as e:
        # 이미 루프가 도는 중이라면 바로 await 시도
        await main()

In [ ]:
# %pip install nest_asyncio

In [ ]:
import os
import asyncio
import json
import random
import nest_asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. 주피터 노트북 환경 설정
nest_asyncio.apply()

# 2. 모델 설정 (gpt-4o-mini는 TPM 제한이 엄격하므로 속도 조절이 필수입니다)
llm = ChatOpenAI(model="gpt-5-nano", temperature=0.7)
OUTPUT_FILE = 'final_compact_dataset2.json'

# 3. 페르소나 정의
PERSONA_EXAMPLES = """
[참고 페르소나 예시]
1. 20대 취업준비생: 대중교통으로 매일 학원에 통원하며, 식비를 아끼기 위해 편의점에서 끼니를 해결함.
2. 30대 여성 직장인: 해외여행을 좋아하며, 병원을 많이 이용하고 자차를 소지함.
3. 20대 사회초년생: 자취 중이라 공과금 부담이 크며, 배달 앱과 스트리밍 구독을 즐김.
4. 40대 학부모: 자녀 학원비와 마트 장보기가 주 지출이며 주유비 절감을 원함.
5. 30대 자기계발러: 운동 센터 결제가 잦고 온라인 쇼핑과 카페 이용이 많음.
"""

# 4. 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", f"당신은 카드 혜택 데이터 요약 및 질문 생성 전문가입니다.\n{PERSONA_EXAMPLES}"),
    ("user", """아래 [혜택 정보]를 보고 질문과 '요약된 정답' 쌍을 생성하세요.

[혜택 정보]: {benefit_content}

조건:
1. anchor: 제시된 페르소나를 참고하여 아주 구체적인 상황이 담긴 질문을 만드세요.
2. positive: 위 혜택 정보에서 '핵심 혜택'과 '주요 조건(전월실적 등)'만 추출하여 200~300자 내외로 요약하세요. 불필요한 문구(에디터 정보 등)는 제거하세요.
3. 결과는 반드시 아래 JSON 형식으로만 답변하세요.
{{
    "anchor": "페르소나 질문",
    "positive": "요약된 핵심 혜택 내용"
}}
""")
])

# 5. [추가] 재시도 로직이 포함된 개별 호출 함수
async def invoke_with_retry(chain, content, max_retries=5):
    for i in range(max_retries):
        try:
            # 60초 타임아웃 적용하여 개별 요청이 무한 대기하는 것 방지
            return await asyncio.wait_for(chain.ainvoke({"benefit_content": content}), timeout=60.0)
        except Exception as e:
            if "rate_limit" in str(e).lower() or "429" in str(e):
                # 지수 백오프: 2, 4, 8, 16초... + 랜덤 노이즈
                wait_time = (2 ** (i + 1)) + random.uniform(0, 1)
                print(f"⏳ Rate Limit 발생! {wait_time:.2f}초 후 재시도... ({i+1}/{max_retries})")
                await asyncio.sleep(wait_time)
            else:
                # 일반 에러(파싱 등)는 상위로 던짐
                raise e
    return None

# 6. 비동기 배치 처리 함수
async def process_batch(batch_chunks, all_data):
    chain = prompt | llm
    # 개별 항목마다 재시도 로직(invoke_with_retry)을 적용
    tasks = [invoke_with_retry(chain, b['content']) for b in batch_chunks]
    responses = await asyncio.gather(*tasks, return_exceptions=True)
    
    batch_results = []
    for idx, res in enumerate(responses):
        if res is None or isinstance(res, Exception): 
            continue
        try:
            raw_content = res.content.strip().replace('```json', '').replace('```', '')
            ai_data = json.loads(raw_content)
            meta = batch_chunks[idx]['metadata']
            
            same_card_others = [
                item['content'] for item in all_data 
                if item['metadata']['card_id'] == meta['card_id'] and item['metadata']['category'] != meta['category']
            ]
            
            raw_neg = random.choice(same_card_others) if same_card_others else "이 질문과 직접적인 관련이 없는 혜택입니다."
            negative_text = raw_neg[:300] + "..." if len(raw_neg) > 300 else raw_neg

            batch_results.append({
                "anchor": ai_data['anchor'],
                "positive": ai_data['positive'],
                "negative": negative_text,
                "metadata": {
                    "card_name": meta['card_name'],
                    "category": meta['category'],
                    "card_id": meta['card_id'],
                    "corp": meta.get('corp', '알 수 없음')
                }
            })
        except:
            continue
    return batch_results

# 7. 메인 실행 함수
async def main():
    # 데이터 로드
    files = ['all_credit_benefits_rag.json', 'all_check_benefits_rag.json']
    all_benefits = []
    for f in files:
        if os.path.exists(f):
            with open(f, 'r', encoding='utf-8') as f_in:
                all_benefits.extend(json.load(f_in))

    final_dataset = []
    if os.path.exists(OUTPUT_FILE):
        try:
            with open(OUTPUT_FILE, 'r', encoding='utf-8') as f_out:
                final_dataset = json.load(f_out)
                print(f"🔄 기존 데이터 발견: {len(final_dataset)}건 이후부터 재개합니다.")
        except:
            pass

    processed_indices = len(final_dataset)
    
    # ✅ [수정] 배치 사이즈를 줄여서 TPM 부하 감소 (20 -> 10)
    batch_size = 10 

    print(f"🚀 총 {len(all_benefits)}건 중 {len(all_benefits) - processed_indices}건 처리를 시작합니다.")

    for i in range(processed_indices, len(all_benefits), batch_size):
        batch = all_benefits[i:i+batch_size]
        
        try:
            # 배치당 타임아웃은 넉넉하게 200초 설정
            results = await asyncio.wait_for(process_batch(batch, all_benefits), timeout=200.0)
            final_dataset.extend(results)
            
            with open(OUTPUT_FILE, 'w', encoding='utf-8') as out:
                json.dump(final_dataset, out, ensure_ascii=False, indent=2)
            
            print(f"💾 {min(i + batch_size, len(all_benefits))} / {len(all_benefits)} 완료")
            
            # ✅ [수정] 배치 완료 후 강제 휴식 시간 부여 (TPM 안정화)
            await asyncio.sleep(1.0) 
            
        except Exception as e:
            print(f"❌ 루프 내부 에러: {e}. 5초 대기 후 다음 배치 시도...")
            await asyncio.sleep(5)
            continue

    print(f"✨ 작업 완료! 파일: {OUTPUT_FILE}")

if __name__ == "__main__":
    await main()